<a href="https://colab.research.google.com/github/ParthikB/Neural-Networks-from-Scratch-2/blob/NN-v1.0/Neural%20Network.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


##############################################

def create_data():
  height = [130, 140, 135, 125, 120]
  weight = [35 ,  30,  30,  35,  35]
  y      = [  0,   1,   0,   1,   1]    
  BMI    = ["Healthy", "Unhealthy", "Healthy", "Unhealthy", "Unhealthy"]

  return np.array([height, weight]), np.array(y).reshape(1, -1)
  

def sigmoid(z):
  A =  1 / (1 + np.exp(-z))
  cache = z
  return A, cache


def sigmoid_derivative(dA, cache):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ


def network_structure(X, neurons_in_hidden_layer, y):
  n_x = X.shape[0]
  n_h = neurons_in_hidden_layer
  n_y = y.shape[0]
  return (n_x, n_h, n_y)
  

def initialize_random_parameters(layer_dims, X):
    parameters = {}
    parameters["W1"] = np.random.randn(layer_dims[0], X.shape[0]) * 0.01
    parameters["b1"] = np.zeros((layer_dims[0], 1))
    for i in range(1, len(layer_dims)):
        parameters["W" + str(i+1)] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        parameters["b" + str(i+1)] = np.zeros((layer_dims[i], 1))

    return parameters

### Basic Neuron Working

In [4]:
# Defining main Class.
class Neuron:
  
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias   = bias
    
    
  def feedforward(self, X):
    # Weights inputs, add bias and then use activation function
    math_e_magic = np.dot(self.weights, X) + self.bias
    output = sigmoid(math_e_magic)
    
    return output
    
    
# Testing
X = [2, 3]
weights, bias = [2, 3], [4]

# > Defining the Class
neuron = Neuron(weights, bias)

# > Feedforwarding
print(neuron.feedforward(X))
# Result --> array([0.99999996])

[0.99999996]


### Neural Network Class

In [0]:
class NeuralNetwork:
  '''
      A Neural Network with:
        - 2 inputs
        - a hidden layer with 2 Neurons
        - an output layer with 1 Neuron
        
  '''    
    
#   def feedforward(self, X, parameters):
#     W1, b1, W2, b2 = parameters
    
#     z1 = np.dot(W1, X) + b1
#     a1 = sigmoid(z1)
    
#     z2 = np.dot(W2, a1) + b2
#     a2 = sigmoid(z2)
    
#     y_hat = a2
#     return y_hat


  def feedforward(self, X, parameters):

    def linear_forward(A, W, b):
      Z = np.dot(W, A) + b
      linear_cache = (A, W, b)

      A, activation_cache = sigmoid(Z)

      cache = (linear_cache, activation_cache)
      return A, cache

    caches = []
    A = X
    L = len(parameters) // 2

    for i in range(1, L+1):
      A, cache = linear_forward(A, parameters["W"+str(i)], parameters["b"+str(i)])
      caches.append(cache)

    return A, caches

  
  def cost(self, yhat, y):
    m = y.shape[1]
    cost = -np.sum(y * np.log(yhat) + (1-y) * np.log(1-yhat)) / m
    return cost
  
  
  def backward_propagation(self, yhat, y, caches):

    def linear_backward(dA, cache):
      linear_cache, activation_cache = cache
      A, W, b = linear_cache

      dZ = sigmoid_derivative(dA, activation_cache)  
      A_prev = A
      m = A_prev.shape[1]

      dW = np.dot(dZ, A_prev.T) / m
      db = np.sum(dZ, axis=1, keepdims=True) / m
      dA_prev = np.dot(W.T, dZ)

      return dA_prev, dW, db


    L = len(caches)
    grads = {}
    y = y.reshape(yhat.shape)
    dyhat = -(np.divide(y, yhat) - np.divide(1-y, 1-yhat))
    grads["dA" + str(L)] = dyhat
    
    for l in range(L)[::-1]:
        current_cache = caches[l]
        grads["dA" + str(l)], grads["dW" + str(l+1)], grads["db" + str(l+1)] = linear_backward(grads["dA" + str(l+1)], current_cache)

    return grads
  
  
  def gradient_descent(parameters, grads):
    

In [0]:
X, y = create_data()
structure = network_structure(X, 2, y)
paras = parameters(structure)

In [0]:
nn = NeuralNetwork()

yhat, caches = nn.feedforward(X, paras)
cost = nn.cost(yhat, y)
grads = nn.backward_propagation(yhat, y, caches)